In [1]:
!pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 9.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 8.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 9.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 6.8 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from tqdm import tqdm
import os
from google.colab import drive

# KcBERT로 모델 수정
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 하이퍼파라미터 설정
MAX_LEN = 256  # 텍스트의 최대 길이
BATCH_SIZE = 32  # 배치 크기
EPOCHS = 5  # 에포크 수
LEARNING_RATE = 2e-5  # 학습률, ex) 1e-3, 1e-4, 2e-5

# # 현재 작업 디렉토리 가져오기
# current_dir = os.getcwd()
# # CSV 파일 경로 설정
# train_csv_file_path = os.path.join(current_dir, '..', '..', 'data', 'processed', 'KcELECTRA_review_train_set.csv')
# test_csv_file_path = os.path.join(current_dir, '..', '..', 'data', 'processed', 'KcELECTRA_review_test_set.csv')

# 구글 드라이브 마운트
drive.mount('/content/drive')

train_csv_file_path = '/content/drive/MyDrive/KoBert_model/ai/data/processed/KcELECTRA_review_train_set_v2.03.csv'
test_csv_file_path = '/content/drive/MyDrive/KoBert_model/ai/data/processed/KcELECTRA_review_test_set_v2.03.csv'

In [5]:
# 리뷰 데이터를 처리하기 위한 커스텀 Dataset 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.review_text = dataframe.Review_Text
        self.targets = dataframe.Label
        self.max_len = max_len

    def __len__(self):
        return len(self.review_text)

    def __getitem__(self, index):
        review = str(self.review_text[index])
        target = self.targets[index]

        # 리뷰 텍스트를 토큰화하고, 패딩 및 트렁케이션 적용
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,  # 특수 토큰 추가 ([CLS], [SEP] 등)
            max_length=self.max_len,  # 최대 길이 제한
            padding='max_length',  # 패딩 적용
            truncation=True,  # 길이를 초과하는 텍스트는 자름
            return_attention_mask=True,
            return_tensors='pt',  # 파이토치 텐서로 반환
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

# 데이터셋 로드
train_df = pd.read_csv(train_csv_file_path, encoding='utf-8-sig')
test_df = pd.read_csv(test_csv_file_path, encoding='utf-8-sig')

# Dataset 생성
train_dataset = ReviewDataset(train_df, tokenizer, MAX_LEN)
test_dataset = ReviewDataset(test_df, tokenizer, MAX_LEN)

# DataLoader를 통해 데이터셋을 배치 단위로 처리
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# 디바이스 설정: GPU가 사용 가능하면 GPU로, 그렇지 않으면 CPU로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델을 디바이스로 이동
model = model.to(device)

# AdamW 옵티마이저와 교차 엔트로피 손실 함수를 사용
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss().to(device)

# 모델 학습 함수
# 하나의 에포크 동안 모델을 학습시키는 함수 정의
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()  # 모델을 학습 모드로 설정
    losses = []

    for batch in tqdm(data_loader):  # 데이터로더에서 배치를 반복적으로 가져옴
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        # 모델의 출력과 손실 계산
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, targets)

        losses.append(loss.item())

        # 역전파를 통해 손실에 대한 그래디언트를 계산하고 가중치를 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return sum(losses) / len(losses)  # 에포크 동안의 평균 손실 반환

# 각 에포크에 대해 모델을 학습
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    print(f'Train loss: {train_loss}')

Epoch 1/3


  0%|          | 3/2701 [00:08<2:12:00,  2.94s/it]


KeyboardInterrupt: 

In [ ]:
# 모델을 평가하는 함수 정의
def eval_model(model, data_loader, device):
    model = model.eval()  # 모델을 평가 모드로 설정
    predictions, targets = [], []

    with torch.no_grad():  # 평가 중에는 그래디언트를 계산하지 않음
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target = batch['targets'].to(device)

            # 모델의 출력 계산
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)  # 가장 높은 로짓 값을 가진 클래스를 예측

            predictions.extend(preds.cpu())  # CPU로 이동하여 리스트에 추가
            targets.extend(target.cpu())  # CPU로 이동하여 리스트에 추가

    # 리스트를 텐서로 변환
    predictions = torch.stack(predictions)
    targets = torch.stack(targets)

    # 정확도, 정밀도, 재현율, F1-score 계산
    accuracy = accuracy_score(targets, predictions)
    precision = precision_score(targets, predictions, average='weighted')
    recall = recall_score(targets, predictions, average='weighted')
    f1 = f1_score(targets, predictions, average='weighted')

    return accuracy, precision, recall, f1

# 테스트 데이터셋에 대한 모델의 평가
accuracy, precision, recall, f1 = eval_model(model, test_loader, device)
print(f'Test Accuracy: {accuracy}')
print(f'Test Precision: {precision}')
print(f'Test Recall: {recall}')
print(f'Test F1 Score: {f1}')

In [ ]:
# 저장할 경로를 지정
model_save_path = "/content/KcELECTRA_model/KcELECTRA_sentiment_model_v"

# 모델 가중치와 설정 저장
model.save_pretrained(model_save_path)

# 토크나이저 설정 저장
tokenizer.save_pretrained(model_save_path)

In [ ]:
from google.colab import files

!zip -r KcELECTRA_sentiment_model_v2.02?????.zip KcELECTRA_sentiment_model_v2.02??????/

files.download('KcELECTRA_sentiment_model_v2.02.zip')